In [10]:
import json

from torch.nn import BCEWithLogitsLoss
from torch.optim.adam import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

from genbot.data import IntentClassificationDataset, IntentPredictionDataset
from genbot.model import IntentClassifier, IntentPredictor

DATASET_FILENAME = '../data/clean/customer_support_twitter_sample.json'
TESTSET_FILENAME = '../data/clean/customer_support_twitter_sample_test.json'
N_EPOCHS = 2

ImportError: cannot import name 'IntentPredictor' from 'genbot.model' (D:\Documents\masters\src\genbot\model.py)

In [2]:
with open(DATASET_FILENAME) as file:
    data = json.load(file)
classifier_dataset = IntentClassificationDataset(data)
classifier_dataloader = DataLoader(classifier_dataset, batch_size=2)
classifier = IntentClassifier(classifier_dataset.n_labels).to('cuda')
classifier_criterion = BCEWithLogitsLoss()
classifier_optimizer = Adam(classifier.parameters(), lr=1e-05)
classifier.train()
for epoch in tqdm(range(N_EPOCHS)):
    print("Epoch", epoch)
    running_loss = 0.
    for inputs, targets in classifier_dataloader:
        outputs = classifier(inputs)
        targets = targets.to('cuda')
        loss = classifier_criterion(outputs, targets)
        running_loss += loss
        loss.backward()
        classifier_optimizer.step()
    print('running_loss:', running_loss/len(classifier_dataset))



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 0


 50%|█████     | 1/2 [00:01<00:01,  1.87s/it]

running_loss: tensor(0.3275, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 1


100%|██████████| 2/2 [00:02<00:00,  1.14s/it]

running_loss: tensor(0.2482, device='cuda:0', grad_fn=<DivBackward0>)


In [5]:
with open(TESTSET_FILENAME) as file:
    test = json.load(file)
classifier_testset = IntentClassificationDataset(test, intents=classifier_dataset.intents)
classifier_testloader = DataLoader(classifier_testset, batch_size=2)
classifier.eval()
for inputs, targets in classifier_testloader:
    outputs = classifier(inputs)
    targets = targets.to('cuda')

    loss = classifier_criterion(outputs, targets)
    print(loss)

tensor(0.3083, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.3289, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [16]:
from torch.nn import Module

class IntentPredictor(Module):

    def __init__(self):
        super(IntentPredictor, self).__init__()

    def forward(self, inputs):
        pass


prediction_dataset = IntentPredictionDataset(data)
prediction_dataloader = DataLoader(prediction_dataset, batch_size=2)
predictor = IntentPredictor().to('cuda')
predictor.train()
for epoch in tqdm(range(N_EPOCHS)):
    print("Epoch", epoch)
    for inputs, targets in classifier_dataloader:
        print(inputs)
        print(targets)

100%|██████████| 2/2 [00:00<00:00, 272.74it/s]

Epoch 0
('USERNAME @AppleSupport Me too am suffering , hope the can find a solution', 'USERNAME We can help. Which version of iOS are you on? You can find that in Settings > General > About. Reply in DM. URL')
tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 1.]])
('I just updated my phone and suddenly everything takes ages to load wtf USERNAME this update sux I hate it fix it bye', 'USERNAME Thanks for reaching out to us. We are always happy to help. Send us a DM so we can look into this together. URL')
tensor([[0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.]])
('@AppleSupport after the 11.0.2 my phone just sucks most of the apps are broken, wifi disconnects frequently #apple #ios1102 #painfulupdate', "USERNAME We'd love to help! Please DM us and let us know any steps you've tried since experiencing these behaviors. URL")
tensor([[0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.]])
